In [5]:
# Célula 1 - Importações e conexão ao banco

import psycopg2
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import StandardScaler

conn = psycopg2.connect(
    dbname="mimiciv",
    user="uti_user",
    password="s0f4C1#4",
    host="localhost",
    port="5432"
)
print("✅ Conexão estabelecida.")


✅ Conexão estabelecida.


DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE mimiciv REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [6]:
# Célula 2 - Criar coorte UTI > 1h

cursor = conn.cursor()
cursor.execute("ROLLBACK")
cursor.execute("""
CREATE TEMP TABLE todas_utis AS
SELECT 
    i.subject_id,
    i.hadm_id,
    i.stay_id,
    i.intime,
    i.outtime,
    EXTRACT(EPOCH FROM (i.outtime - i.intime)) / 60 AS duracao_minutos
FROM mimiciv_icu.icustays i
WHERE EXTRACT(EPOCH FROM (i.outtime - i.intime)) > 3600
  AND i.stay_id IN (
      SELECT DISTINCT stay_id
      FROM mimiciv_derived.vasoactive_agent
  )
LIMIT 50;
""")
conn.commit()

df_total = pd.read_sql("SELECT COUNT(*) AS total_admissoes FROM todas_utis;", conn)
print(f"✅ Total admissões na UTI (>1h): {df_total['total_admissoes'][0]}")


✅ Total admissões na UTI (>1h): 50


/tmp/ipykernel_51965/829476087.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_total = pd.read_sql("SELECT COUNT(*) AS total_admissoes FROM todas_utis;", conn)


In [7]:
query_vitals = """
SELECT
    vs.stay_id,
    vs.charttime,
    vs.heart_rate,
    vs.sbp,
    vs.dbp,
    vs.mbp,
    vs.resp_rate,
    vs.temperature,
    vs.spo2,
    vs.glucose
FROM mimiciv_derived.vitalsign vs
WHERE vs.stay_id IN (SELECT stay_id FROM todas_utis)
ORDER BY vs.stay_id, vs.charttime;
"""

df_vitals = pd.read_sql(query_vitals, conn)
df_vitals['charttime'] = pd.to_datetime(df_vitals['charttime'])
df_vitals = df_vitals.sort_values(['stay_id', 'charttime'])

df_5min = (
    df_vitals
    .set_index('charttime')
    .groupby('stay_id', group_keys=False)
    .resample('5min')
    .mean()
    .reset_index()
)

df_5min['stay_id'] = df_5min['stay_id'].ffill()

sinais_vitais = ['heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature', 'spo2', 'glucose']

def calcular_intervalos_amostragem(df, variaveis, tempo_col='charttime', id_col='stay_id'):
    parametros = {}
    for var in variaveis:
        diffs_totais = []
        for _, g in df.groupby(id_col):
            tempos = g[tempo_col].dropna().sort_values()
            if len(tempos) > 1:
                diffs = tempos.diff().dt.total_seconds().dropna()
                diffs_totais.extend(diffs.values)
        if diffs_totais:
            mediana = np.median(diffs_totais)
            iqr = np.percentile(diffs_totais, 75) - np.percentile(diffs_totais, 25)
            parametros[var] = (mediana, iqr)
        else:
            parametros[var] = (3600, 1800)
    return parametros

NORMAL_VALUES = {
    'heart_rate': 80,
    'sbp': 120,
    'dbp': 80,
    'mbp': 90,
    'resp_rate': 16,
    'temperature': 37,
    'spo2': 98,
    'glucose': 100,
}

parametros = calcular_intervalos_amostragem(df_5min, sinais_vitais)

def imputacao_adaptativa(df, variaveis, parametros, normal_values, tempo_col='charttime', id_col='stay_id'):
    df = df.sort_values([id_col, tempo_col]).reset_index(drop=True)
    df_imputado = df.copy()
    for var in variaveis:
        mediana, iqr = parametros.get(var, (3600, 1800))
        max_forward_sec = mediana + iqr
        valor_mediano = np.nanmedian(df_imputado[var])
        if np.isnan(valor_mediano):
            valor_mediano = 0.0
        for stay_id, grupo in df_imputado.groupby(id_col):
            g = grupo.sort_values(tempo_col).reset_index()
            tempos = g[tempo_col]
            valores = g[var].values
            imputed_vals = []
            mascara = []
            ultimo_valor = None
            ultimo_tempo = None
            for i in range(len(g)):
                if not np.isnan(valores[i]):
                    imputed_vals.append(valores[i])
                    mascara.append(1)
                    ultimo_valor = valores[i]
                    ultimo_tempo = tempos.iloc[i]
                else:
                    if ultimo_valor is None:
                        imputed_vals.append(normal_values.get(var, valor_mediano))
                        mascara.append(0)
                    else:
                        delta = (tempos.iloc[i] - ultimo_tempo).total_seconds()
                        if delta <= max_forward_sec:
                            imputed_vals.append(ultimo_valor)
                            mascara.append(0)
                        elif delta <= 2 * max_forward_sec:
                            frac = (delta - max_forward_sec) / max_forward_sec
                            val = (1 - frac) * ultimo_valor + frac * valor_mediano
                            imputed_vals.append(val)
                            mascara.append(0)
                        else:
                            imputed_vals.append(valor_mediano)
                            mascara.append(0)
            df_imputado.loc[g['index'], var] = imputed_vals
            df_imputado.loc[g['index'], f'{var}_mask'] = mascara
    return df_imputado

df_5min = imputacao_adaptativa(df_5min, sinais_vitais, parametros, NORMAL_VALUES)
print("✅ Sinais vitais imputados com método adaptativo do HiRID e máscaras criadas.")



/tmp/ipykernel_51965/3020859792.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vitals = pd.read_sql(query_vitals, conn)


✅ Sinais vitais imputados com método adaptativo do HiRID e máscaras criadas.


In [8]:
itemids_labs = [
    50861, 50862, 53085, 50908, 51580, 50883, 50884, 50885, 50910, 50924,
    50963, 50915, 52642, 51002, 51003, 50889, 52116, 51623, 50928, 52117,
    51214, 50878, 50855, 50912, 52546, 53161, 53180, 52142, 51265, 51266,
    52144, 50931, 50935, 51631, 51638, 51640, 51222, 51223, 50856, 51647,
    50852, 51643, 50971, 50983, 50990, 50967, 50968, 50969, 50960, 50966,
    50970, 50975, 51099, 51006, 51274, 51275, 51292, 51290, 51291, 50963,
    51196, 52551, 50915, 51568, 51569, 51570, 51464, 51966, 50803, 50805,
    50808, 50809, 50813
]

query_labs = f"""
SELECT 
    icu.stay_id,
    le.charttime,
    le.itemid,
    le.valuenum
FROM mimiciv_hosp.labevents le
JOIN mimiciv_icu.icustays icu
  ON le.subject_id = icu.subject_id AND le.hadm_id = icu.hadm_id
WHERE le.itemid IN ({','.join(map(str, itemids_labs))})
  AND icu.stay_id IN (SELECT stay_id FROM todas_utis)
  AND le.valuenum IS NOT NULL
ORDER BY icu.stay_id, le.charttime;
"""

df_labs = pd.read_sql(query_labs, conn)
df_labs['charttime'] = pd.to_datetime(df_labs['charttime'])

df_labs_pivot = (
    df_labs
    .pivot_table(index=['stay_id', 'charttime'], columns='itemid', values='valuenum', aggfunc='mean')
    .reset_index()
)

df_labs_pivot.columns.name = None
df_labs_pivot = df_labs_pivot.rename(columns={itemid: f"lab_{itemid}" for itemid in df_labs['itemid'].unique()})

df_merged = pd.merge(df_5min, df_labs_pivot, how='left', on=['stay_id', 'charttime'])

lab_vars = [col for col in df_merged.columns if col.startswith('lab_')]

for var in lab_vars:
    # Máscara de presença (1=original, 0=imputado)
    df_merged[f'{var}_mask'] = (~df_merged[var].isna()).astype(int)

    # Interpolação linear para preencher pequenos gaps
    df_merged[var] = (
        df_merged
        .sort_values(['stay_id', 'charttime'])
        .groupby('stay_id')[var]
        .transform(lambda g: g.interpolate(method='linear', limit_direction='both'))
    )

    # Forward-fill limitado a 24h (288 janelas de 5 minutos)
    df_merged[var] = (
        df_merged
        .sort_values(['stay_id', 'charttime'])
        .groupby('stay_id')[var]
        .transform(lambda g: g.ffill(limit=288))
    )

    # Preenchimento inicial e final com mediana observada da variável
    median_val = df_merged[var].median()
    df_merged[var] = (
        df_merged
        .groupby('stay_id')[var]
        .transform(lambda g: g.fillna(median_val if not np.isnan(median_val) else 0))
    )

print("✅ Exames laboratoriais imputados com interpolação, forward-fill limitado e preenchimento inicial pela mediana.")

/tmp/ipykernel_51965/2046256324.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_labs = pd.read_sql(query_labs, conn)
/home/grad/si/22/aliciachaves/miniconda3/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/grad/si/22/aliciachaves/miniconda3/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/grad/si/22/aliciachaves/miniconda3/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


✅ Exames laboratoriais imputados com interpolação, forward-fill limitado e preenchimento inicial pela mediana.


In [9]:
# Célula 5 - Extrair vasopressores e marcar falência (opção 2: merge por stay_id)

query_vasoact = """
SELECT 
    stay_id,
    starttime,
    endtime,
    dopamine,
    epinephrine,
    norepinephrine,
    phenylephrine,
    vasopressin,
    dobutamine,
    milrinone
FROM mimiciv_derived.vasoactive_agent
WHERE stay_id IN (SELECT stay_id FROM todas_utis);
"""

df_vasoact = pd.read_sql(query_vasoact, conn)
df_vasoact['starttime'] = pd.to_datetime(df_vasoact['starttime'])
df_vasoact['endtime'] = pd.to_datetime(df_vasoact['endtime'])

vaso_cols = ['dopamine', 'epinephrine', 'norepinephrine', 'phenylephrine',
             'vasopressin', 'dobutamine', 'milrinone']

# Indicador de uso de qualquer vasopressor
df_vasoact['vasopressor_ativo'] = df_vasoact[vaso_cols].notna().any(axis=1)

# Garantir tipos corretos
df_merged['stay_id'] = df_merged['stay_id'].astype(int)
df_merged['charttime'] = pd.to_datetime(df_merged['charttime'])
df_vasoact['stay_id'] = df_vasoact['stay_id'].astype(int)
df_vasoact['starttime'] = pd.to_datetime(df_vasoact['starttime'])

# Remover valores NaT se houver
df_merged = df_merged.dropna(subset=['charttime'])
df_vasoact = df_vasoact.dropna(subset=['starttime'])

# Ordenar dentro dos grupos
df_merged = df_merged.sort_values(['stay_id', 'charttime'], kind='mergesort').reset_index(drop=True)
df_vasoact = df_vasoact.sort_values(['stay_id', 'starttime'], kind='mergesort').reset_index(drop=True)

# Lista para acumular os merges por grupo
dfs_merged = []

# Loop para merge por stay_id
for sid in df_merged['stay_id'].unique():
    df_merged_sid = df_merged[df_merged['stay_id'] == sid].sort_values('charttime')
    df_vasoact_sid = df_vasoact[df_vasoact['stay_id'] == sid].sort_values('starttime')

    if df_vasoact_sid.empty:
        # Nenhum vaso para esse stay_id, apenas adiciona df_merged com NaNs
        df_merged_sid['starttime'] = pd.NaT
        df_merged_sid['endtime'] = pd.NaT
        for col in vaso_cols:
            df_merged_sid[col] = pd.NA
        dfs_merged.append(df_merged_sid)
        continue

    merged_sid = pd.merge_asof(
        df_merged_sid,
        df_vasoact_sid[['stay_id', 'starttime', 'endtime'] + vaso_cols],
        left_on='charttime',
        right_on='starttime',
        by='stay_id',
        direction='backward',
        tolerance=pd.Timedelta('2D')
    )
    dfs_merged.append(merged_sid)

# Concatenar todos os grupos
df_vaso_merged = pd.concat(dfs_merged, ignore_index=True)

print("\n✅ Merge por stay_id realizado com sucesso.")

# Marcar falência circulatória
cond_tempo = (df_vaso_merged['charttime'] >= df_vaso_merged['starttime']) & \
             (df_vaso_merged['charttime'] <= df_vaso_merged['endtime'])
cond_vaso = df_vaso_merged[vaso_cols].notna().any(axis=1)
cond_mbp = df_vaso_merged['mbp'] < 65
cond_lactato = df_vaso_merged.get('lab_50813', pd.Series(0)) >= 2

df_vaso_merged['falencia'] = 0
indices_falencia = df_vaso_merged.index[cond_tempo & cond_vaso & (cond_mbp | cond_lactato)]
df_vaso_merged.loc[indices_falencia, 'falencia'] = 1

print(f"✅ Falência circulatória marcada. Total: {df_vaso_merged['falencia'].sum()}")



/tmp/ipykernel_51965/1154267511.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vasoact = pd.read_sql(query_vasoact, conn)



✅ Merge por stay_id realizado com sucesso.
✅ Falência circulatória marcada. Total: 7332


In [10]:
# Célula 7 - Criar features de instabilidade

def criar_features_instabilidade(df, event_col='falencia'):
    df = df.sort_values(['stay_id', 'charttime']).reset_index(drop=True)
    df['estado_atual'] = df[event_col]

    df['tempo_desde_ultimo_evento'] = np.nan

    for stay_id, grupo in df.groupby('stay_id'):
        indices = grupo.index
        estados = grupo['estado_atual'].values
        tempos = grupo['charttime'].values

        last_event_time = None
        tempo_desde = []

        for i, estado in enumerate(estados):
            if estado == 1:
                last_event_time = tempos[i]
                tempo_desde.append(0)
            else:
                if last_event_time is None:
                    tempo_desde.append(np.nan)
                else:
                    delta = (tempos[i] - last_event_time).astype('timedelta64[m]').astype(float)
                    tempo_desde.append(delta)

        df.loc[indices, 'tempo_desde_ultimo_evento'] = tempo_desde

    df['duracao_evento'] = df.groupby('stay_id')['estado_atual'].transform(lambda x: x.expanding().mean())

    return df

df_vaso_merged = criar_features_instabilidade(df_vaso_merged)
print("✅ Features de instabilidade criadas.")



✅ Features de instabilidade criadas.


In [11]:
# Célula 8 - Criar features de intensidade de medição

def criar_features_intensidade(df, vars_continuas):
    df = df.sort_values(['stay_id', 'charttime']).reset_index(drop=True)

    # Dicionário para armazenar todas as colunas novas temporariamente
    novas_colunas = {}

    for var in vars_continuas:
        mask = ~df[var].isna()

        # Arrays temporários para armazenar valores antes do merge
        tempo_desde_ultima_medicao = np.full(len(df), np.nan)
        prop_medicoes = np.full(len(df), np.nan)

        for stay_id, grupo in df.groupby('stay_id'):
            indices = grupo.index
            tempos = grupo['charttime'].values
            mask_var = mask.loc[indices].values

            last_meas_time = None
            tempos_desde = []
            contagem = 0

            for i, presente in enumerate(mask_var):
                if presente:
                    last_meas_time = tempos[i]
                    contagem += 1
                    tempos_desde.append(0)
                else:
                    if last_meas_time is None:
                        tempos_desde.append(np.nan)
                    else:
                        delta = (tempos[i] - last_meas_time).astype('timedelta64[m]').astype(float)
                        tempos_desde.append(delta)

            prop_medicoes_grupo = [contagem / (i+1) for i in range(len(tempos))]

            tempo_desde_ultima_medicao[indices] = tempos_desde
            prop_medicoes[indices] = prop_medicoes_grupo

        novas_colunas[f'{var}_tempo_desde_ultima_medicao'] = tempo_desde_ultima_medicao
        novas_colunas[f'{var}_prop_medicoes'] = prop_medicoes

    # Criar DataFrame das novas colunas e concatenar tudo de uma vez
    df_novas = pd.DataFrame(novas_colunas, index=df.index)
    df = pd.concat([df, df_novas], axis=1)

    return df


vars_continuas = [col for col in df_vaso_merged.columns if col not in ['stay_id', 'charttime', 'falencia'] and not col.endswith('_mask')]
df_vaso_merged = criar_features_intensidade(df_vaso_merged, vars_continuas)
print("✅ Features de intensidade de medição criadas.")



✅ Features de intensidade de medição criadas.


In [12]:
vars_continuas = sinais_vitais + lab_vars  # lista completa dos seus sinais vitais e labs

vars_validas = [var for var in vars_continuas if df_vaso_merged[var].isna().mean() < 0.5]
masks_validas = [v + '_mask' for v in vars_validas]

colunas_finais = vars_validas + masks_validas + ['stay_id', 'charttime', 'falencia']

df_final = df_vaso_merged[colunas_finais].copy()

print(f"✅ Dataset filtrado com {len(vars_validas)} variáveis contínuas válidas (<50% missing).")


✅ Dataset filtrado com 46 variáveis contínuas válidas (<50% missing).


In [15]:
def construir_janelas_temporais(df, jan_obs=36, jan_pred=12, passo=1, max_nan_ratio=0.5):
    # Seleciona colunas ignorando ids, tempo, label e máscaras
    candidate_cols = [col for col in df.columns if col not in ['stay_id', 'charttime', 'falencia'] and not col.endswith('_mask')]
    
    # Filtra somente colunas numéricas para evitar problemas com NaT ou objetos
    vars_features = [col for col in candidate_cols if pd.api.types.is_numeric_dtype(df[col])]
    
    if len(vars_features) == 0:
        raise ValueError("Nenhuma variável numérica válida para construir janelas.")
    
    X, y, stays, times = [], [], [], []

    total_janelas = 0
    rejeitadas_nan = 0

    for stay_id, group in df.groupby('stay_id'):
        group = group.reset_index(drop=True)
        count_validas = 0
        print(f"\nAnalisando stay_id {stay_id} com {len(group)} registros")

        for i in range(0, len(group) - (jan_obs + jan_pred) + 1, passo):
            janela_obs = group.iloc[i : i + jan_obs]
            janela_pred = group.iloc[i + jan_obs : i + jan_obs + jan_pred]

            nan_ratio = janela_obs[vars_features].isna().mean().mean()
            total_janelas += 1

            if nan_ratio > max_nan_ratio:
                rejeitadas_nan += 1
                print(f"Janela {i} rejeitada por NaN ({nan_ratio:.2%} > {max_nan_ratio:.2%})")
                continue

            X.append(janela_obs[vars_features].values)
            y.append(1 if janela_pred['falencia'].any() else 0)
            stays.append(stay_id)
            times.append(janela_obs['charttime'].iloc[0])
            count_validas += 1

        print(f"stay_id {stay_id} tem {count_validas} janelas válidas")

    print(f"\nTotal janelas avaliadas: {total_janelas}")
    print(f"Janelas rejeitadas por NaN: {rejeitadas_nan} ({rejeitadas_nan/total_janelas:.2%})")

    return np.array(X), np.array(y), stays, times


# Teste
X, y, stays, times = construir_janelas_temporais(df_vaso_merged, max_nan_ratio=0.5)
print(f"\n✅ Janelas temporais criadas. Total: {len(X)}, Formato X: {X.shape}, y: {y.shape}")





Analisando stay_id 30121250 com 997 registros
stay_id 30121250 tem 950 janelas válidas

Analisando stay_id 30155942 com 177 registros
stay_id 30155942 tem 130 janelas válidas

Analisando stay_id 30713595 com 1860 registros
stay_id 30713595 tem 1813 janelas válidas

Analisando stay_id 30974510 com 1154 registros
stay_id 30974510 tem 1107 janelas válidas

Analisando stay_id 31038553 com 1021 registros
stay_id 31038553 tem 974 janelas válidas

Analisando stay_id 31059962 com 901 registros
stay_id 31059962 tem 854 janelas válidas

Analisando stay_id 31646998 com 334 registros
stay_id 31646998 tem 287 janelas válidas

Analisando stay_id 31754815 com 3469 registros
stay_id 31754815 tem 3422 janelas válidas

Analisando stay_id 31904119 com 561 registros
stay_id 31904119 tem 514 janelas válidas

Analisando stay_id 32032568 com 178 registros
stay_id 32032568 tem 131 janelas válidas

Analisando stay_id 32280626 com 1552 registros
stay_id 32280626 tem 1505 janelas válidas

Analisando stay_id 324

In [17]:
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

X_shape = X.shape  # (amostras, janela_obs, features)
X_flat = X.reshape(-1, X_shape[2])  # achata para (amostras * janela_obs, features)

# Tratar NaNs e infinitos
X_flat = np.nan_to_num(X_flat, nan=0.0, posinf=0.0, neginf=0.0)
X_flat = X_flat.astype(np.float64)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat).reshape(X_shape)

X = X_scaled

joblib.dump(scaler, "scaler.pkl")

print("✅ Padronização z-score aplicada.")



✅ Padronização z-score aplicada.


In [18]:
# Célula 11 - Exportar dados para arquivos (opcional)

np.save("X.npy", X)
np.save("y.npy", y)
pd.DataFrame({"stay_id": stays, "start_time": times}).to_csv("janelas_metadata.csv", index=False)
print("✅ Dados exportados.")



✅ Dados exportados.
